In [2]:
#Importing the libraries
import bs4
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib2
import re


# Teams

In [8]:
#Getting the url for all the pages on the site
teURLs ='https://sofifa.com/teams?r=200040&set=true&offset='
fullsitet=range(0,1000,60)
sofifatl=[]
for i in fullsitet:
    
    sofifatl.append(teURLs+ str(i))
print sofifatl

['https://sofifa.com/teams?r=200040&set=true&offset=0', 'https://sofifa.com/teams?r=200040&set=true&offset=60', 'https://sofifa.com/teams?r=200040&set=true&offset=120', 'https://sofifa.com/teams?r=200040&set=true&offset=180', 'https://sofifa.com/teams?r=200040&set=true&offset=240', 'https://sofifa.com/teams?r=200040&set=true&offset=300', 'https://sofifa.com/teams?r=200040&set=true&offset=360', 'https://sofifa.com/teams?r=200040&set=true&offset=420', 'https://sofifa.com/teams?r=200040&set=true&offset=480', 'https://sofifa.com/teams?r=200040&set=true&offset=540', 'https://sofifa.com/teams?r=200040&set=true&offset=600', 'https://sofifa.com/teams?r=200040&set=true&offset=660', 'https://sofifa.com/teams?r=200040&set=true&offset=720', 'https://sofifa.com/teams?r=200040&set=true&offset=780', 'https://sofifa.com/teams?r=200040&set=true&offset=840', 'https://sofifa.com/teams?r=200040&set=true&offset=900', 'https://sofifa.com/teams?r=200040&set=true&offset=960']


In [9]:
#Using beautiful soup to get team data
ofn=sofifatl
tb=[]
hre=[]
leag=[]
lb=[]
for of in ofn:
    teaURLs = of
    pagest = requests.get(teaURLs)
    soupt = BeautifulSoup(pagest.text, 'html.parser')
    for tr in soupt.find_all('tr'):
        leag.append(tr.find('div' ,class_="bp3-text-overflow-ellipsis",text=True))
        hre.append(tr.find('a').get('href'))
        ts=tr.find_all('td')
        for t in ts:
            tbo=t.get_text()
            tb.append(tbo)
        


In [10]:
te=[]
for l in leag:
    if l==None:
        pass
    else:
        te.append(l.get_text())

In [11]:
#Cleaning up the data
ctb=[x.encode('latin1', 'ignore').decode('utf8', 'ignore') for x in tb]
nstb=[x.split('\n') for x in ctb]
tbdf=pd.DataFrame(nstb,columns=['num','temd','c','d','e'])
tbdfd=(tbdf.temd.dropna())
tbdfn=(tbdfd[~tbdfd.str.isnumeric()]) #Removing non numeric values
fteams=tbdf.iloc[1::9, :].drop(columns=['num','c','d','e'])
fteams=fteams.reset_index()
fteams=fteams.drop(columns=['index'])
#Getting the overall rating
ova=tbdf.iloc[2::9, :].drop(columns=['temd','c','d','e'])
ova=ova.reset_index()
ova=ova.drop(columns=['index'])
#Getting the attacking rating
att=tbdf.iloc[3::9, :].drop(columns=['temd','c','d','e'])
att=att.reset_index()
att=att.drop(columns=['index'])
#Getting the midfield rating
mid=tbdf.iloc[4::9, :].drop(columns=['temd','c','d','e'])
mid=mid.reset_index()
mid=mid.drop(columns=['index'])
#Getting the defence rating
defs=tbdf.iloc[5::9, :].drop(columns=['temd','c','d','e'])
defs=defs.reset_index()
defs=defs.drop(columns=['index'])
#Getting each team transfer budget
bud=tbdf.iloc[6::9, :].drop(columns=['temd','c','d','e'])
bud=bud.reset_index()
bud=bud.drop(columns=['index'])
hres=['https://sofifa.com'+x for x in hre]
#Removing unnessecary values
href=[x for x in hres if x!='https://sofifa.com/teams?r=200040&set=true&col=oa&sort=desc']
hrefs=pd.DataFrame(href,columns=['url'])
#Putting it all together in the dataframe
sft=pd.concat([fteams,ova,att,mid,defs,bud,hrefs],axis=1)
sft.columns=['Team','Overall','Attack','Midfield','Defence','Budget','URL']
#print sft
print sft

                                                   Team Overall Attack  \
0          Manchester United English Premier League (1)      81     83   
1            Borussia Dortmund German 1. Bundesliga (1)      82     83   
2             FC Bayern Mnchen German 1. Bundesliga (1)      84     87   
3            Manchester City English Premier League (1)      85     87   
4                  Liverpool English Premier League (1)      85     87   
5                    Arsenal English Premier League (1)      80     84   
6                Real Madrid Spain Primera Division (1)      86     85   
7               FC Barcelona Spain Primera Division (1)      86     88   
8                             Inter Italian Serie A (1)      82     83   
9                    Chelsea English Premier League (1)      81     80   
10                            Milan Italian Serie A (1)      79     85   
11         Bayer 04 Leverkusen German 1. Bundesliga (1)      80     80   
12                  RB Leipzig German 

In [12]:
to=[x.encode('latin1', 'ignore').decode('utf8', 'ignore') for x in te]
leagues=[]
for t,s in zip(to,sft['Team']):
   # print [x.replace(t, ' ') for x in s]
    r=re.compile(t)
    tc=r.sub('',s)
    leagues.append(tc)
#print leagues

sft['League']=pd.DataFrame((leagues))
print leagues

[u' English Premier League (1)', u' German 1. Bundesliga (1)', u' German 1. Bundesliga (1)', u' English Premier League (1)', u' English Premier League (1)', u' English Premier League (1)', u' Spain Primera Division (1)', u' Spain Primera Division (1)', u' Italian Serie A (1)', u' English Premier League (1)', u' Italian Serie A (1)', u' German 1. Bundesliga (1)', u' German 1. Bundesliga (1)', u' French Ligue 1 (1)', u' English Premier League (1)', u' Spain Primera Division (1)', u' German 1. Bundesliga (1)', u' French Ligue 1 (1)', u' Italian Serie A (1)', u' Spain Primera Division (1)', u' Holland Eredivisie (1)', u' English Premier League (1)', u' German 1. Bundesliga (1)', u' Italian Serie A (1)', u' English Premier League (1)', u' Italian Serie A (1)', u' English Premier League (1)', u' French Ligue 1 (1)', u' French Ligue 1 (1)', u' English Premier League (1)', u' Spain Primera Division (1)', u' Portuguese Liga ZON SAGRES (1)', u' German 1. Bundesliga (1)', u' Portuguese Liga ZON S

In [13]:
teamo=[]
for t,s in zip(to,sft['Team']):
   # print [x.replace(t, ' ') for x in s]
    r=re.compile(s)
    to=r.sub('',t)
    teamo.append(to)
#print leagues

sft['Teams']=pd.DataFrame((teamo))
print sft

                                                   Team Overall Attack  \
0          Manchester United English Premier League (1)      81     83   
1            Borussia Dortmund German 1. Bundesliga (1)      82     83   
2             FC Bayern Mnchen German 1. Bundesliga (1)      84     87   
3            Manchester City English Premier League (1)      85     87   
4                  Liverpool English Premier League (1)      85     87   
5                    Arsenal English Premier League (1)      80     84   
6                Real Madrid Spain Primera Division (1)      86     85   
7               FC Barcelona Spain Primera Division (1)      86     88   
8                             Inter Italian Serie A (1)      82     83   
9                    Chelsea English Premier League (1)      81     80   
10                            Milan Italian Serie A (1)      79     85   
11         Bayer 04 Leverkusen German 1. Bundesliga (1)      80     80   
12                  RB Leipzig German 

In [14]:
#Using the team url to get the prestige of each team
hress=[x for x in hre if x!='/teams?r=200040&set=true&col=oa&sort=desc'] # removing unnessecary parts
ip=[]
dp=[]
turl=[]
tid=[]
for hr in hress:
    trURLs ='https://sofifa.com'+ str(hr)
    pagesr= requests.get(trURLs)
    soupr = BeautifulSoup(pagesr.text, 'html.parser')
    span=soupr.find_all('span',class_='bp3-tag bp3-minimal p')
    spans=([x.text.strip() for x in span])
    spans=[x.encode("utf-8") for x in spans]
    ip.append(spans)
    urll=(trURLs)
    urlls=trURLs.split('/')[5]
    tid.append(urlls) #Team ID
    turl.append(urll) # Team URL


mscteams=pd.DataFrame(zip(turl,tid,ip))
#mscteams.columns=['URL','ID','pres']
#print mscteams

In [15]:
#cleaning up the dataframe and preparing to combine the dataframes
sofifateams=pd.read_csv('fifa20_teams.csv')
sofifateams.columns=['in','URL','Team','pres']
sofifateams=sofifateams.drop(columns=['in'])
sofifateams['Team']=sofifateams.Team.str.replace('-',' ')
sofifateams['ID']=sofifateams.URL.str.split('/').str[4]
ip=sofifateams['pres'].str[7:10]
ip=ip.str.replace('\'',' ')
ip=ip.str.replace(']',' ')
ip=ip.str.replace(' ','')
ip=ip.str.replace(',','')
fip=ip.to_frame()
fip.columns=['Iprestige']
dp=sofifateams['pres'].str[2:5]
dp=dp.str.replace('\'',' ')
dp=dp.str.replace(',',' ')
fdp=dp.to_frame()
fdp.columns=['Dprestige']
finalteams=pd.concat([sofifateams,fdp,fip],axis=1)
fifa20_teamsf=finalteams.drop(columns=['pres'])

print display(fifa20_teamsf)

,URL,Team,ID,Dprestige,Iprestige
0,https://sofifa.com/team/22/borussia-dortmund/,borussia dortmund,22,7,9
1,https://sofifa.com/team/32/bayer-04-leverkusen/,bayer 04 leverkusen,32,6,7
2,https://sofifa.com/team/21/fc-bayern-munchen/,fc bayern munchen,21,9,10
3,https://sofifa.com/team/11/manchester-united/,manchester united,11,7,8
4,https://sofifa.com/team/241/fc-barcelona/,fc barcelona,241,10,10
5,https://sofifa.com/team/1/arsenal/,arsenal,1,7,7
6,https://sofifa.com/team/243/real-madrid/,real madrid,243,10,10
7,https://sofifa.com/team/5/chelsea/,chelsea,5,7,8
8,https://sofifa.com/team/9/liverpool/,liverpool,9,10,9
9,https://sofifa.com/team/34/fc-schalke-04/,fc schalke 04,34,6,8


None


In [34]:
#putting it all togethter and cleaning up the team dataset
sofifateams=pd.read_csv('cfifa20_teams.csv')
sofifallt=(pd.merge(sofifateams,sft,on='URL'))
sofifallt=sofifallt.drop(columns=['Unnamed: 0','Team_y'])
sofifallt=sofifallt.rename(columns={"Team_x":"Team"})
sofifallt['Budget']=sofifallt.Budget.str.replace('M','000000')
sofifallt['Budget']=sofifallt.Budget.str.replace('K','000')
sofifallt['Budget']=sofifallt.Budget.str.replace('.','')
#sofifallt=sofifallt.drop(columns=['Team'])
print display(sofifallt)
sofifallt.to_csv('finalteamdata_fifa2020.csv')

,URL,Team,ID,Dprestige,Iprestige,Overall,Attack,Midfield,Defence,Budget,League,Teams
0,https://sofifa.com/team/22/borussia-dortmund/,borussia dortmund,22,7,9.0,82,83,83,82,60000000,German 1. Bundesliga (1),Borussia Dortmund
1,https://sofifa.com/team/32/bayer-04-leverkusen/,bayer 04 leverkusen,32,6,7.0,80,80,80,79,39000000,German 1. Bundesliga (1),Bayer 04 Leverkusen
2,https://sofifa.com/team/21/fc-bayern-munchen/,fc bayern munchen,21,9,10.0,84,87,85,81,100000000,German 1. Bundesliga (1),FC Bayern Mnchen
3,https://sofifa.com/team/11/manchester-united/,manchester united,11,7,8.0,81,83,81,81,177000000,English Premier League (1),Manchester United
4,https://sofifa.com/team/241/fc-barcelona/,fc barcelona,241,10,10.0,86,88,85,85,1879000000,Spain Primera Division (1),FC Barcelona
5,https://sofifa.com/team/1/arsenal/,arsenal,1,7,7.0,80,84,82,78,74000000,English Premier League (1),Arsenal
6,https://sofifa.com/team/243/real-madrid/,real madrid,243,10,10.0,86,85,87,85,1885000000,Spain Primera Division (1),Real Madrid
7,https://sofifa.com/team/5/chelsea/,chelsea,5,7,8.0,81,80,84,81,91000000,English Premier League (1),Chelsea
8,https://sofifa.com/team/9/liverpool/,liverpool,9,10,9.0,85,87,83,85,103000000,English Premier League (1),Liverpool
9,https://sofifa.com/team/34/fc-schalke-04/,fc schalke 04,34,6,8.0,77,76,77,76,42000000,German 1. Bundesliga (1),FC Schalke 04


None


# Player

In [ ]:
#Getting the url for all the pages in the site
URLs ='https://sofifa.com/players?r=200039&set=true&offset='
fullsite=range(0,19660,60)
sofifal=[]
for i in fullsite:
    
    sofifal.append(URLs+ str(i))
print sofifal

In [ ]:
#Webscrape player data

coun=[]
name=[]
ids=[]
pos=[]
age=[]
rat=[]
pot=[]
clubs=[]
conc=[]
val=[]
wage=[]
ur=[]
ul=[]
for tx in sofifal:
    URL=tx
    pages = requests.get(URL)
    soup = BeautifulSoup(pages.text, 'html.parser')
    #player data
    table_body=soup.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]
        cols=[x.encode('latin1', 'ignore').decode('utf8', 'ignore') for x in cols]
        htag=(row.find('a').get('href'))
        ur.append(htag)
        age.append(cols[2])#2 age
        rat.append(cols[3])#3 rating
        pot.append(cols[4])#4 potential
        conc.append(club[1]) #contract and or loan status
        val.append(cols[6]) # value
        wage.append(cols[7]) # wage

In [ ]:
#combining the webscraped player data
cpl=pd.DataFrame((zip(age,rat,pot,clubs,conc,val,wage,ur)))
cpl.columns=['Age','Rating','Potential','Club','Contract_length','Current_Value','Current_Wage','url']
cpl['Current_Value']=cpl.Current_Value.str.replace('M','000000')
cpl['Current_Value']=cpl.Current_Value.str.replace('K','000')
cpl['Current_Wage']=cpl.Current_Wage.str.replace('K','000')
print display(cpl)

In [ ]:
#Combining the player dataframe together
#Reading and cleaning the data to prepare it for the analysis
bpd= pd.read_csv('players_20.csv')

print display(bpd.head())